## imports

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig
import torch
from torch.cuda.amp import autocast, GradScaler
import time

start_all = time.time()

In [2]:
peft_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_rslora=False)

In [3]:
quantization_config = BitsAndBytesConfig(
        # load_in_8bit=True,
        # load_in_4bit=True,
        llm_int8_enable_fp32_cpu_offload=True,
        # llm_int8_has_fp16_weight=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype="float16"
)

In [4]:
device = 'cuda'

In [5]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config=quantization_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [7]:
#from datasets import load_dataset
#dataset = load_dataset("pszemraj/booksum-short")

In [8]:
#text = dataset['train']['chapter'][0]

In [9]:
with open("data/books/J._K._Rowling_-_Harry_Potter_1_-_Sorcerers_Stone.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [10]:
print(len(tokenizer.encode(text, return_tensors='pt')[0]))

127505


## Fonctions

In [14]:
def enlever_inst(contenu):
    # Trouver la position de la balise [/INST]
    fin_inst = contenu.find('[/INST]')

    # Vérifier si la balise [/INST] a été trouvée
    if fin_inst != -1:
        # Supprimer tout le texte avant et y compris la balise [/INST]
        resultat = contenu[fin_inst + len('[/INST]'):]
        return resultat
    else:
        # Si la balise [/INST] n'a pas été trouvée, retourner le contenu original
        return contenu


In [15]:
# Diviser le texte en segments de 500 caractères ou moins
# mettre 8000
def split_text_into_token_segments(text, token_amount=5000):
    segments = []

    encoded_text = tokenizer.encode(text=text, return_tensors='pt')

    sequence=[]
    for tensor in encoded_text[0]:
        sequence.append(tensor)

        if len(sequence) == token_amount:
            segments.append(sequence)
            sequence = []

    if sequence :
        segments.append(sequence)

    return segments

In [16]:
def summarize_token_segment(segments, instructions = 'make me a summary of the following text: '):

    summaries = []
    counter = 0
    for segment in segments:
        print(counter)
        counter += 1
        text_to_summarize = tokenizer.decode(segment, skip_special_tokens=True)
        message = [
            {
                "role":"user",
                "content": f"{instructions}"
                           f"{text_to_summarize}"
            }
        ]

        encodeds = tokenizer.apply_chat_template(message, return_tensors="pt").to(device)

        generated_ids = model.generate(encodeds, do_sample=True, max_new_tokens=1000, pad_token_id=tokenizer.pad_token_id)
        decoded = tokenizer.batch_decode(generated_ids)

        summaries.append([enlever_inst(decoded[0])])

    return summaries

In [17]:
def group_summaries(summaries):
    grouped_summaries = ''.join([item for sublist in summaries for item in sublist])
    return split_text_into_token_segments(grouped_summaries)

In [18]:
def full_process(text):
    token_segments = split_text_into_token_segments(text)
    while True:
        print('a')
        summary = summarize_token_segment(token_segments)
        print('b')
        grouped_summaries = group_summaries(summary)
        print('c')
        if len(grouped_summaries) == 1:
            return summarize_token_segment(grouped_summaries)
        else:
            token_segments = grouped_summaries

In [19]:
start = time.time()
test = full_process(text)
end = time.time()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


a
0


C:\Users\devar\Desktop\ESGI\master\M1\S2\PA-4A\venv\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

b
c
a
0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


b
c
0


In [20]:
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

The time of execution of above program is : 1611119.2829608917 ms


In [21]:
end_all = time.time()

print("The time of execution of the entire program is :",
      (end_all-start_all) * 10**3, "ms")

The time of execution of the entire program is : 1710936.838388443 ms


In [22]:
test

[[' In the opening chapter of "Harry Potter and the Sorcerer\'s Stone," Harry Potter, a muggle child living with his muggle relatives, the Dursleys, experiences strange occurrences and dreams of being a wizard. Mr. Dursley dismisses these events as tricks of the mind. Meanwhile, Albus Dumbledore and Professor McGonagall discuss Voldemort\'s return and the rumors about the Potter family. Hagrid, the Keeper of Keys and Grounds at Hogwarts, arrives to take Harry to the magical school, revealing his true identity as a wizard.\n\nHarry, Hagrid, and Dumbledore go to Diagon Alley to buy school supplies, and Harry purchases a wand, which chooses him. They meet Hermione Granger and set off for Hogwarts on the Hogwarts Express. Harry is excited to start his new life at Hogwarts but is met with challenges as they suspect Professor Snape of attempting to steal the Sorcerer\'s Stone, Harry\'s first Quidditch match approaches, and the Christmas holidays arrive.\n\nWhile searching for information on 

In [23]:
message = [
            {
                "role":"user",
                "content": f"make me a short summary of the following text without being to precise :"
                           f"{test}"
            }
        ]

encodeds = tokenizer.apply_chat_template(message, return_tensors="pt").to(device)

generated_ids = model.generate(encodeds, do_sample=True, max_new_tokens=1000, pad_token_id=tokenizer.pad_token_id)
decoded = tokenizer.batch_decode(generated_ids)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [25]:
enlever_inst(decoded[0])

' In the first chapter of "Harry Potter and the Sorcerer\'s Stone," Harry, a muggle boy living with muggle relatives, experiences strange occurrences and dreams of being a wizard. He is later taken to Hogwarts, a magical school, by Hagrid, where he meets Ron and Hermione and starts his new life. They encounter various challenges, including suspicious behavior from Professor Snape and the sorcerer\'s stone being stolen. Harry discovers his invisibility cloak and faces off against Malfoy. The group continues to uncover the truth about the sorcerer\'s stone whilst dealing with fear, isolation, and the dangers of the Mirror of Erised. Along the way, they meet several key characters and come across various magical elements such as wands, Quidditch, and the Hogwarts Express. Key events include the revelation of Harry\'s wizarding identity, the suspicion surrounding Snape, and their excitement for Quidditch and Christmas.</s>'